# Задание на пятую лабораторную:
Реализовать вопросно ответную систему типа "retriever-reader"
Напоминаю идею: ридер получает на вход текст и вопрос к нему, и возвращает место в тексте, которое является ответом на вопрос. Ретривер ищет из списка текстов тот, в котором с наибольшей вероятностью найдется ответ на вопрос (по косинусной мере между вопросом и каждым текстом) -- вот это нужно реализовать. Вектора текстов для ретривера можете генерировать, как вам больше нравится -- трансформеры/word2vec/tf-idf или что-то еще
На выходе должно быть приблизительно следующее:
1. Функция-ретривер: получает на вход тексты и вопрос, возвращает нужный текст
2. Функция-ридер: получает текст и вопрос, возвращает ответ
3. Функция, внутри которой последовательно вызывается ретривер и ридер
4. Добавьте, пожалуйста, несколько примеров работы вашего кода, чтобы было видно, что вы передали тексты и вопрос, и получили правильный ответ


# Библиотеки

In [2]:
import json
import pandas as pd
import torch
import numpy as np
from typing import List, Union
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from sentence_transformers import SentenceTransformer, util
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine
from typing import List, Union
import datasets

# Параметры

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Функция-ретривер

In [4]:
class Retriever:
    def __init__(self, MODEL):
        global device
        self.model_ = BertModel.from_pretrained(MODEL, output_hidden_states=True)
        self.model_.eval()  # Устанавливаем модель в режим оценки
        self.model_.to(device)  # Перемещаем модель на указанное устройство
        self.tokenizer_ = BertTokenizer.from_pretrained(MODEL, do_lower_case=True)  # Инициализация токенизатора
    
    # Преобразование текста в векторное представление
    def vectors_from_texts(self, text):
        marked_text = "[CLS] " + text + " [SEP]"
        tokenized_text = self.tokenizer_.tokenize(marked_text)
        if len(tokenized_text) > 512:
            marked_text = "[CLS] " + text + " [SEP]"
            tokenized_text = self.tokenizer_.tokenize(marked_text)

        indexed_tokens = self.tokenizer_.convert_tokens_to_ids(tokenized_text)

        segments_ids = [1] * len(tokenized_text)

        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])
        tokens_tensor = tokens_tensor.to(device)
        segments_tensors = segments_tensors.to(device)

        with torch.no_grad():
            outputs = self.model_(tokens_tensor, segments_tensors)
            hidden_states = outputs[2]

        token_embeddings = torch.stack(hidden_states, dim=0)
        token_embeddings = torch.squeeze(token_embeddings, dim=1)
        token_embeddings = token_embeddings.permute(1, 0, 2)
        token_vecs_sum = []
        for token in token_embeddings:
            sum_vec = torch.sum(token[-4:], dim=0)
            token_vecs_sum.append(sum_vec)

            token_vecs = hidden_states[-2][0]

        sentence_embedding = torch.mean(token_vecs, dim=0)
        return sentence_embedding.cpu().numpy()
    
    # Получение наиболее близкого текста к вопросу с использованием косинусного расстояния
    def get_nearest_text(self, texts: List[str], question: str):
        context_vectors = []
        for paragraph in texts:
            context_vectors.append(self.vectors_from_texts(paragraph))

        question_vector = self.vectors_from_texts(question)
        result = 1
        result_id = 0
        counter = 0
        for vector in context_vectors:
            if cosine(vector, question_vector) < result:
                result = cosine(vector, question_vector)
                result_id = counter
            counter += 1

        return texts[result_id]

In [5]:
retriever = Retriever('DeepPavlov/rubert-base-cased')

c:\Users\emely\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\emely\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Функция-ридер

In [6]:
class Reader:
    def __init__(self, model, tokenizer):
        self.model_ = model
        self.tokenizer_ = tokenizer

    # Получение ответа на вопрос относительно заданного текста с использованием модели
    def get_answer(self, text: str, question: str):
        # Подготовка входных данных для модели с использованием токенизатора
        encoding = self.tokenizer_.encode_plus(text=question, text_pair=text)
        inputs = encoding['input_ids']
        tokens = self.tokenizer_.convert_ids_to_tokens(inputs)
        # Передача входных данных модели и получение выхода
        output = self.model_(input_ids=torch.tensor([inputs]))
        start_index = torch.argmax(output[0])
        end_index = torch.argmax(output[1])

        # Формирование ответа на основе токенов
        answer = " ".join(tokens[start_index:end_index+1])
        answer = answer.replace(" ", "")[1:].split("▁")
        answer = " ".join(answer)

        return answer

In [7]:
tokenizer = AutoTokenizer.from_pretrained("AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru")
model = AutoModelForQuestionAnswering.from_pretrained("AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru")
reader = Reader(model, tokenizer)



# Функция, внутри которой последовательно вызывается ретривер и ридер

In [8]:
class MyModel:
    def __init__(self, retriever, reader):
        self.retriever_ = retriever
        self.reader_ = reader

    def get_answer(self, texts: Union[str, List[str]], question: str):
        if isinstance(texts, str):
            answer = self.reader_.get_answer(texts, question)
            return answer

        sim_text = self.retriever_.get_nearest_text(texts, question)
        answer = self.reader_.get_answer(sim_text, question)
        return answer

In [9]:
qa_model = MyModel(retriever, reader)

# Добавьте, пожалуйста, несколько примеров работы вашего кода, чтобы было видно, что вы передали тексты и вопрос, и получили правильный ответ

In [10]:
dataset = datasets.load_dataset("sberquad")

In [15]:
def check_qa(model, dataset, start_idx, end_idx, check_text_idx):
    texts = dataset["context"][start_idx:end_idx]
    question = dataset["question"][check_text_idx]

    true_answer = dataset["answers"][check_text_idx]["text"][0]
    pred_answer = model.get_answer(texts, question)

    print(f"True Answer is:       {true_answer}")
    print(f"Predicted Answer is:  {pred_answer}")
    print()

texts = dataset["train"]
for i in [10, 11, 2]:
    check_qa(qa_model, texts, 0, 10, i)

True Answer is:       2050
Predicted Answer is:  истекает договор аренды Байконура

True Answer is:       9 млрд рублей в год
Predicted Answer is:  

True Answer is:       органические остатки
Predicted Answer is:  органические остатки



In [18]:
question = "Сколько окенанов на Земле?"
texts = [
    """
    Нил — одна из самых важных и древних рек в мире, играющая ключевую роль в истории и развитии региона. 
    Её истоки находятся в Восточной Африке, где сходятся две её основные притоки: Белый Нил и Голубой Нил. 
    Белый Нил начинается из озера Виктория, а Голубой Нил из озера Танганьика. Объединившись в Хартуме, столице Судана, они образуют Нил.
    Нил протекает через несколько стран, включая Уганду, Кения, Эфиопию, Судан, и Египет. В Египте он впадает 
    в Средиземное море, создавая дельту Нила. Эта река была жизненно важной для древних цивилизаций, таких как древний 
    Египет, предоставляя воду для полива и сельского хозяйства.
    Современные гидроэлектростанции, такие как Асуанская плотина в Египте, сделали Нил также важным источником энергии. 
    Однако регулирование потока реки также вызывает различные вопросы в области экологии и устойчивого развития.
    Несмотря на свою длину и значимость, бассейн Нила также стал местом напряженности в связи с использованием его воды 
    между странами, протекающими вдоль его берегов.
    """,
    """
    Океаны Земли составляют около 99% общего объема жидкости на поверхности планеты. 
    Всего четыре океана - Тихий, Атлантический, Индийский и Северный Ледовитый.
    """,
    """
    Длинное слово в английском языке без использования каких-либо химических терминов - "pneumonoultramicroscopicsilicovolcanoconiosis". 
    Это название заболевания легких, вызванного вдыханием мельчайших кремниевых частиц.
    """
]

qa_model.get_answer(texts, question)

'Всего четыре'

In [22]:
question = "Какая река была жизненно важной для древних цивилизаций?"
qa_model.get_answer(texts, question)

'Нил'

In [23]:
question = "Какое самое длиннок слово в английском языке?"
qa_model.get_answer(texts, question)

'neumonoultramicroscopicsilicovolcanoconiosis".'